# Convolutional Networks

**Convolutional networks** also known as CNNs, are specialized neural networks for processing data that has a known grid-like topology. Convolutional networks are simply neural networks that use convolution in place of general matrix multiplication in at least one of their layers. A **convolution** is a specialized kind of liner operation.